In [ ]:
# Read in the data
RE = read.csv("RE.csv")
str(RE)

In [ ]:
#Linear regression without covariate

Regression1 = lm(Life~Drug,data=RE)
summary(Regression1)



In [ ]:
#linear regression with the covariate adjusted health

Regression2 = lm(Life~Drug+AdjustedHealth,data=RE)
summary(Regression2)



In [ ]:
#linear regression with the interaction term

Regression3 = lm(Life~Drug+AdjustedHealth+Drug*AdjustedHealth,data=RE)
summary(Regression3)

In [ ]:
##Install and load the package for propensity-score matching

install.packages('Matching',repos = "https://cloud.r-project.org")
library('Matching')
library(caTools)



In [ ]:
# Fit a logistic regression to estimate the propensity score
pscore_model = glm(Drug ~ AdjustedHealth, data=RE, family=binomial)

pscore = predict(pscore_model,type="response")


RE$pscore = pscore

Drug = RE$Drug

Life = RE$Life

In [ ]:
#Run propensity score matching and estimate the average treatment effect on treated
match.pscore = Match(Y =Life, Tr=Drug, X=pscore, estimand = "ATT", M = 1)

summary(match.pscore)

In [ ]:
##Balance Check
MatchBalance(Drug~AdjustedHealth,data=RE,match.out=match.pscore)

In [ ]:
#install and load the necessary packages for IV estimation
install.packages("AER",repos = "https://cloud.r-project.org")
library("AER")

In [ ]:
#read in the data
Education = read.csv("Education.csv")
str(Education)

In [ ]:
#linear regression model
model1=lm(LogEarning~Schooling,data=Education)
summary(model1)

In [ ]:
#transform QoB into factor variable
Education$QoBF=as.factor(Education$QoB)

#Relevel QoBF
Education$QoBF=relevel(Education$QoBF,"1")

str(Education)

In [ ]:
#first-stage regression X~Z and compute X_hat
model_fs = lm(Schooling~QoBF,data=Education)
Education$Schooling1 = predict(model_fs,newdata=Education)
summary(model_fs)
##F-test is significant, so the strong first-stage assumption holds.

In [ ]:
#second-stage regression Y~X_hat
Model2 = lm(LogEarning~Schooling1,data=Education)
summary(Model2)

In [ ]:
#2sls combined estimation
Model3 = ivreg(LogEarning~Schooling|QoBF,data=Education)
summary(Model3)